In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
import os
import json
import matplotlib.pyplot as plt
from scipy.stats import linregress
%matplotlib inline

# Import the API key.
from config import g_key
gmaps.configure(api_key = g_key)

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Satipo,PE,2020-10-24 22:46:09,-11.25,-74.64,82.09,51,66,2.26
1,1,Atuona,PF,2020-10-24 22:46:09,-9.80,-139.03,78.44,76,1,21.90
2,2,Nikolskoye,RU,2020-10-24 22:46:09,59.70,30.79,37.99,86,40,2.24
3,3,Stornoway,GB,2020-10-24 22:46:10,58.21,-6.39,48.20,81,40,31.09
4,4,Barrow,US,2020-10-24 22:46:10,71.29,-156.79,33.80,86,90,11.41


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Create Northern and Southern Hemisphere DataFrames.
northern_hemi_df = city_data_df.loc[(city_data_df["Lat"] >= 0)]
southern_hemi_df = city_data_df.loc[(city_data_df["Lat"] < 0)]

In [6]:
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]

In [7]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])

In [8]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

In [9]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [13]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Satipo,PE,2020-10-24 22:46:09,-11.25,-74.64,82.09,51,66,2.26
1,1,Atuona,PF,2020-10-24 22:46:09,-9.80,-139.03,78.44,76,1,21.90
9,9,Avarua,CK,2020-10-24 22:46:11,-21.21,-159.78,77.00,69,60,14.99
25,25,Kirakira,SB,2020-10-24 22:46:16,-10.45,161.92,85.53,69,4,7.74
28,28,Aquidauana,BR,2020-10-24 22:43:45,-20.47,-55.79,82.51,63,17,3.18
31,31,Mataura,NZ,2020-10-24 22:46:17,-46.19,168.86,75.99,48,96,11.01
32,32,Kapaa,US,2020-10-24 22:46:18,22.08,-159.32,82.40,74,40,14.99
35,35,Taoudenni,ML,2020-10-24 22:46:18,22.68,-3.98,87.98,12,0,8.59
38,38,Daru,PG,2020-10-24 22:46:08,-9.08,143.21,81.81,79,98,8.46
39,39,Saint George,US,2020-10-24 22:45:03,37.10,-113.58,80.01,29,1,4.70


In [14]:
preferred_cities_df.count()

City_ID       500
City          500
Country       497
Date          500
Lat           500
Lng           500
Max Temp      500
Humidity      500
Cloudiness    500
Wind Speed    500
dtype: int64

In [15]:
preferred_cities_df.dropna()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Satipo,PE,2020-10-24 22:46:09,-11.25,-74.64,82.09,51,66,2.26
1,1,Atuona,PF,2020-10-24 22:46:09,-9.80,-139.03,78.44,76,1,21.90
9,9,Avarua,CK,2020-10-24 22:46:11,-21.21,-159.78,77.00,69,60,14.99
25,25,Kirakira,SB,2020-10-24 22:46:16,-10.45,161.92,85.53,69,4,7.74
28,28,Aquidauana,BR,2020-10-24 22:43:45,-20.47,-55.79,82.51,63,17,3.18
...,...,...,...,...,...,...,...,...,...,...
1722,1722,Zinder,NE,2020-10-24 22:48:31,13.81,8.99,82.76,15,88,12.59
1733,1733,Abnub,EG,2020-10-24 22:48:34,27.27,31.15,78.80,36,0,11.41
1735,1735,Luganville,VU,2020-10-24 22:48:34,-15.53,167.17,84.20,74,75,8.05
1739,1739,Biak,ID,2020-10-24 22:48:35,-0.91,122.88,80.19,73,100,1.79


In [16]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Satipo,PE,82.09,-11.25,-74.64,
1,Atuona,PF,78.44,-9.80,-139.03,
9,Avarua,CK,77.00,-21.21,-159.78,
25,Kirakira,SB,85.53,-10.45,161.92,
28,Aquidauana,BR,82.51,-20.47,-55.79,
31,Mataura,NZ,75.99,-46.19,168.86,
32,Kapaa,US,82.40,22.08,-159.32,
35,Taoudenni,ML,87.98,22.68,-3.98,
38,Daru,PG,81.81,-9.08,143.21,
39,Saint George,US,80.01,37.10,-113.58,


In [17]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [18]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [20]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [21]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [22]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))